**Импорт библиотек**

In [1]:
import os
import shutil
import json
from PreapreDataLib import create_folder_if_not_exists

from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector
import torch

D:\University\miniconda3\envs\RoadDamageDetection\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
DATASET_DIR = './DatasetCocoFormat/'
TRAIN_DIR = DATASET_DIR + 'train/'
TEST_DIR  = DATASET_DIR + 'test/'
VAL_DIR = DATASET_DIR + 'val/'
ANN_DIR = DATASET_DIR + 'annotations/'
TRAIN_ANN_PTH = ANN_DIR + "train.json"
VAL_ANN_PTH = ANN_DIR + "val.json"
TEST_ANN_PTH = ANN_DIR + "test.json"

**Загрузка конфига Mask R-CNN ResNet101 FPN 1x**

In [3]:
CONFIGS_DIR = "./configs/"
create_folder_if_not_exists("./configs/")
MRCNN_BASE_CONFIG_PTH = CONFIGS_DIR + "mask_rcnn_r50_fpn_1x_coco.py"
MRCNN_USER_CONFIG_PTH = CONFIGS_DIR + "mask_rcnn_r50_fpn_1x_coco_for_road_damages.py"
if not os.path.exists(MRCNN_BASE_CONFIG_PTH):
    !mim download mmdet --config mask_rcnn_r50_fpn_1x_coco --dest "./configs"

**Изменение конфига под задачу и датасет**

In [4]:
cfg = Config.fromfile(MRCNN_BASE_CONFIG_PTH)

# cfg.merge_from_dict(dict(classes = ["lane", "crack", "pothole"]))

cfg.model.roi_head.bbox_head.num_classes = 3
cfg.model.roi_head.mask_head.num_classes = 3

cfg.data_root = DATASET_DIR

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 1

cfg.data.train.img_prefix = TRAIN_DIR
cfg.data.val.img_prefix = VAL_DIR
cfg.data.test.img_prefix = TEST_DIR

cfg.data.train.ann_file = TRAIN_ANN_PTH
cfg.data.val.ann_file = VAL_ANN_PTH
cfg.data.test.ann_file = TEST_ANN_PTH

# (1280, 640)
cfg.test_pipeline[1].scale = (894, 448)
cfg.train_pipeline[2].scale = (894, 448)

cfg.data.train.pipeline[2].img_scale = (894, 448)
cfg.data.val.pipeline[1].img_scale = (894, 448)
cfg.data.test.pipeline[1].img_scale = (894, 448)

cfg.optimizer.lr = 0.001
cfg.optimizer.weight_decay = 0.0001

cfg.classes = ("lane", "crack", "pothole")
cfg.data.train.classes = cfg.classes
cfg.data.val.classes = cfg.classes
cfg.data.test.classes = cfg.classes

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = "cuda"

cfg.work_dir = CONFIGS_DIR

if os.path.exists(MRCNN_USER_CONFIG_PTH):
    os.remove(MRCNN_USER_CONFIG_PTH)
cfg.dump(MRCNN_USER_CONFIG_PTH)

user_config = Config.fromfile(MRCNN_USER_CONFIG_PTH)
print(user_config.pretty_text)

model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)

**Создание модели**

In [5]:
model = build_detector(user_config.model)
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=5.81s)
creating index...
index created!


**Обучение модели**

In [7]:
# model.with_cp = True
# model.fp16_enabled = True
if not os.path.exists(CONFIGS_DIR + "epoch_12.pth"):
    train_detector(model, datasets[0], cfg, distributed=False, validate=True)